In [105]:
from docplex.mp.model import Model
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

**Conjuntos**

$$ \text{CLIENTES} = \{1, 2, \ldots, N\} $$
$$ \text{NODOS} = \{0\} \cup \text{CLIENTES} $$
$$ \text{CAMIONES} = \{1, 2, 3, 4, 5, 6, 7\} $$

In [106]:
N = 20
CLIENTES = list(range(1, N + 1))
NODOS = [0] + CLIENTES
CAMIONES = [1, 2, 3, 4, 5, 6, 7]

In [107]:
random.seed(142)

**Parámetros**

$$ \text{CAPACIDAD\_CAMION}_c = 70 \text{ para cada camión } c \in \text{CAMIONES} $$
$$ \text{DEMANDA}_{cl}: \text{Demanda del cliente } cl \in \text{CLIENTES} $$
$$ \text{DISTANCIAS}_{n1,n2}: \text{Distancia euclidiana entre } n1 \in \text{NODOS} \text{ y } n2 \in \text{NODOS} $$

In [108]:
CAPACIDAD_CAMION = {c: 70 for c in CAMIONES}
DEMANDAS = {c: random.randint(10, 20) for c in CLIENTES}
COORDENADAS = {n: {"x": random.randint(0, N * 10), "y": random.randint(0, N * 10)} for n in NODOS}
DISTANCIAS = {(n1, n2): ((COORDENADAS[n1]["x"] - COORDENADAS[n2]["x"]) ** 2 + (COORDENADAS[n1]["y"] - COORDENADAS[n2]["y"]) ** 2) ** 0.5 for n1 in NODOS for n2 in NODOS}

In [109]:
fig = go.Figure()

# Añadir punto de depósito en rojo
fig.add_trace(
    go.Scatter(
        x=[COORDENADAS[0]["x"]],
        y=[COORDENADAS[0]["y"]],
        mode="markers",
        name="Depósito",
        marker=dict(color="red", size=15),
    )
)

# Añadir puntos de clientes en negro
fig.add_trace(
    go.Scatter(
        x=[coord["x"] for coord in list(COORDENADAS.values())[1:]],
        y=[coord["y"] for coord in list(COORDENADAS.values())[1:]],
        mode="markers",
        name="Clientes",
        marker=dict(color="black", size=10),
    )
)

# Añadir anotaciones de clientes
for cliente in CLIENTES:
    coord_cliente_x = COORDENADAS[cliente]["x"]
    coord_cliente_y = COORDENADAS[cliente]["y"]
    fig.add_annotation(x=coord_cliente_x - 8, y=coord_cliente_y + 8, text=f"<b>C{cliente}: </b> {DEMANDAS[cliente]}", showarrow=False)

# Configurar el layout del gráfico
fig.update_layout(
    title=f"COORDENADAS NODOS",
    xaxis_title="X",
    yaxis_title="Y",
    xaxis=dict(range=[-10, N * 10 + 10]),
    yaxis=dict(range=[-10, N * 10 + 10]),
    width=800,
    height=600,
    template="ggplot2",
)

fig.show()

In [110]:
model = Model("CVRP")

**Variables**

$$ x_{n1, n2, c} = \begin{cases} 
1 & \text{Si el camión } c \in \text{CAMIONES} \text{ viaja de } n1 \in \text{NODOS} \text{ a } n2 \in \text{NODOS} \\ 
0 & \text{d.l.c.} 
\end{cases} $$
$$ u_{n, c} \in \mathbb{R}^{+} \text{: Carga acumulada del vehículo } c \text{ al llegar al nodo } n \in \text{NODOS} $$

In [111]:
x = model.binary_var_dict([(nodo1, nodo2, camion) for nodo1 in NODOS for nodo2 in NODOS for camion in CAMIONES], name="x")
u = model.continuous_var_dict([(nodo, camion) for nodo in NODOS for camion in CAMIONES], name="u")

**Función Objetivo**
$$ \text{Minimizar } \sum_{c \in \text{CAMIONES}} \sum_{n1 \in \text{NODOS}} \sum_{n2 \in \text{NODOS}} \text{DISTANCIAS}_{n1, n2} \cdot x_{n1, n2, c} $$

In [112]:
model.minimize(model.sum(DISTANCIAS[(nodo1, nodo2)] * x[(nodo1, nodo2, camion)] for nodo1 in NODOS for nodo2 in NODOS for camion in CAMIONES))

### Restricciones

1. **Prohibición de bucles:**
$$ \forall n \in \text{NODOS}, \forall c \in \text{CAMIONES}: x_{n, n, c} = 0 $$

2. **Asignación única de salidas para cada cliente:**
$$ \forall cl \in \text{CLIENTES}: \sum_{n \in \text{NODOS}} \sum_{c \in \text{CAMIONES}} x_{cl, n, c} = 1 $$

3. **Asignación única de entradas para cada cliente:**
$$ \forall cl \in \text{CLIENTES}: \sum_{n \in \text{NODOS}} \sum_{c \in \text{CAMIONES}} x_{n, cl, c} = 1 $$

4. **Restricciones para evitar subtours:**
$$ \forall cl1, cl2 \in \text{CLIENTES}, cl1 \neq cl2, \forall c \in \text{CAMIONES}: \text{Si } x_{cl1, cl2, c} = 1 \rightarrow u_{cl2, c} =  u_{cl1, c} + \text{DEMANDA}_{cl2} $$
$$ \forall cl1, cl2 \in \text{CLIENTES}, cl1 \neq cl2, \forall c \in \text{CAMIONES}: u_{cl2, c} \geq u_{cl1, c} + \text{DEMANDA}_{cl2} - \mathcal{M}  \cdot (1 - x_{cl1, cl2, c}) $$


5. **Demanda mínima en cada cliente:**
$$ \forall cl \in \text{CLIENTES}, \forall c \in \text{CAMIONES}: u_{cl, c} \geq \text{DEMANDA}_{cl} $$

6. **Demanda máxima permitida por el camión:**
$$ \forall cl \in \text{CLIENTES}, \forall c \in \text{CAMIONES}: u_{cl, c} \leq \text{CAPACIDAD\_CAMION}_{c} $$

7. **Visita de un camión desde el depósito a un cliente:**
$$ \forall c \in \text{CAMIONES}: \sum_{cl \in \text{CLIENTES}} x_{0, cl, c} \leq 1 $$

8. **Retorno de un camión del cliente al depósito:**
$$ \forall c \in \text{CAMIONES}: \sum_{cl \in \text{CLIENTES}} x_{cl, 0, c} \leq 1 $$

9. **Flujo de continuidad:**
$$ \forall c \in \text{CAMIONES}, \forall n1 \in \text{NODOS}: \sum_{n2 \in \text{NODOS}} x_{n1, n2, c} = \sum_{n2 \in \text{NODOS}} x_{n2, n1, c} $$

In [113]:
for nodo in NODOS:
    for camion in CAMIONES:
        model.add_constraint(x[nodo, nodo, camion] == 0)

for cliente in CLIENTES:
    model.add_constraint(model.sum(x[cliente, nodo, camion] for nodo in NODOS for camion in CAMIONES) == 1)

for cliente in CLIENTES:
    model.add_constraint(model.sum(x[nodo, cliente, camion] for nodo in NODOS for camion in CAMIONES) == 1)

for cliente1 in CLIENTES:
    for cliente2 in CLIENTES:
        for camion in CAMIONES:
            if cliente1 != cliente2:
                model.add_constraint(u[cliente2, camion] >= u[cliente1, camion] + DEMANDAS[cliente2] - 10000 * (1 - x[cliente1, cliente2, camion]))
                # model.add_indicator(x[(cliente1, cliente2)], u[cliente2] == u[cliente1] + DEMANDAS[cliente2], active_value = 1)

for cliente in CLIENTES:
    for camion in CAMIONES:
        model.add_constraint(u[cliente, camion] >= DEMANDAS[cliente])

for cliente in CLIENTES:
    for camion in CAMIONES:
        model.add_constraint(u[cliente, camion] <= CAPACIDAD_CAMION[camion])

for camion in CAMIONES:
    model.add_constraint(model.sum(x[0, cliente, camion] for cliente in CLIENTES) <= 1)
    model.add_constraint(model.sum(x[cliente, 0, camion] for cliente in CLIENTES) <= 1)

for camion in CAMIONES:
    for nodo in NODOS:
        model.add_constraint(model.sum(x[nodo_, nodo, camion] for nodo_ in NODOS) == model.sum(x[nodo, nodo_, camion] for nodo_ in NODOS))

In [114]:
# model.parameters.mip.limits.solutions = 1
model.set_time_limit(200)
solution = model.solve()

In [116]:
rutas = {}
for camion in CAMIONES:
    for cliente in CLIENTES:
        if x[0, cliente, camion].solution_value > 0.9:
            print(f"==== Ruta camión {camion} ====")
            ruta = [0, cliente]
            r = "0 -> " + str(cliente) + " -> "

            cliente_actual = cliente
            while x[cliente_actual, 0, camion].solution_value < 0.9:
                for siguiente_cliente in CLIENTES:
                    if x[cliente_actual, siguiente_cliente, camion].solution_value > 0.9:
                        break
                ruta.append(siguiente_cliente)
                r += str(siguiente_cliente) + " -> "
                cliente_actual = siguiente_cliente

            ruta.append(0)
            r += "0"
            print(r)
            rutas[camion] = ruta
            
print("\n==== Arreglo de rutas ====")
print(rutas)


==== Ruta camión 1 ====
0 -> 18 -> 17 -> 10 -> 15 -> 0
==== Ruta camión 2 ====
0 -> 5 -> 11 -> 14 -> 6 -> 8 -> 0
==== Ruta camión 3 ====
0 -> 3 -> 7 -> 16 -> 13 -> 0
==== Ruta camión 4 ====
0 -> 9 -> 12 -> 2 -> 0
==== Ruta camión 7 ====
0 -> 19 -> 1 -> 4 -> 20 -> 0

==== Arreglo de rutas ====
{1: [0, 18, 17, 10, 15, 0], 2: [0, 5, 11, 14, 6, 8, 0], 3: [0, 3, 7, 16, 13, 0], 4: [0, 9, 12, 2, 0], 7: [0, 19, 1, 4, 20, 0]}


In [117]:
fig = go.Figure()

# RUTAS
for id_camion, ruta in rutas.items():
    coords_x = [COORDENADAS[nodo]["x"] for nodo in ruta]
    coords_y = [COORDENADAS[nodo]["y"] for nodo in ruta]
    capacidad_ruta = sum(map(lambda x: DEMANDAS[x] if x != 0 else 0, ruta))
    distancia_ruta = sum([DISTANCIAS[(ruta[i], ruta[i + 1])] for i in range(len(ruta) - 1)])

    fig.add_trace(
        go.Scatter(
            x=coords_x,
            y=coords_y,
            mode="lines",
            line=dict(color=px.colors.qualitative.G10[id_camion % len(px.colors.qualitative.G10)], width=2, dash="solid"),
            name=f"Ruta C{id_camion} - CAP: {capacidad_ruta} D: {distancia_ruta:.2f}",
        )
    )

    # FLECHAS
    for i in range(len(ruta) - 1):
        mid_x = (coords_x[i] + coords_x[i + 1]) / 2
        mid_y = (coords_y[i] + coords_y[i + 1]) / 2
        fig.add_annotation(
            x=mid_x, y=mid_y,
            ax=mid_x - (coords_x[i + 1] - coords_x[i]) * 0.1, ay=mid_y - (coords_y[i + 1] - coords_y[i]) * 0.1,
            xref="x",
            yref="y",
            axref="x",
            ayref="y",
            showarrow=True,
            arrowhead=1,
            arrowsize=2,
            arrowwidth=1,
            arrowcolor=px.colors.qualitative.G10[id_camion % len(px.colors.qualitative.G10)],
        )

# DEPÓSITO
fig.add_trace(
    go.Scatter(
        x=[COORDENADAS[0]["x"]],
        y=[COORDENADAS[0]["y"]],
        mode="markers",
        name="Depósito",
        marker=dict(color="red", size=15),
    )
)

# CLIENTES
fig.add_trace(
    go.Scatter(
        x=[coord["x"] for coord in list(COORDENADAS.values())[1:]],
        y=[coord["y"] for coord in list(COORDENADAS.values())[1:]],
        mode="markers",
        name="Clientes",
        marker=dict(color="black", size=10),
    )
)

for cliente in CLIENTES:
    coord_cliente_x = COORDENADAS[cliente]["x"]
    coord_cliente_y = COORDENADAS[cliente]["y"]
    fig.add_annotation(x=coord_cliente_x - 8, y=coord_cliente_y + 8, text=f"<b>C{cliente}</b>", showarrow=False)


fig.update_layout(
    title=f"RECORRIDO TOTAL: {solution.objective_value:.2f}",
    xaxis_title="X",
    yaxis_title="Y",
    xaxis=dict(range=[-10, N * 10 + 10]),
    yaxis=dict(range=[-10, N * 10 + 10]),
    width=800,
    height=600,
    template="ggplot2",
)

fig.show()